In [1]:
import torch

In [2]:
x = torch.tensor([-float('inf'), 0])
x

tensor([-inf, 0.])

In [3]:
torch.max(x)

tensor(0.)

In [4]:
x =torch.randn((5,5))
mask = torch.tensor([True]*3 + [False]*2, dtype=torch.bool)


In [64]:
def f(x, mask, n):
    _, indices = torch.sort(
        torch.masked_fill(
            x, torch.logical_not(mask), -float("inf")
        ),
        descending=True,
    )
    return indices

In [65]:
for _ in range(20000):
    xs =torch.randn((2,5,5))
    masks = torch.stack([torch.tensor([True]*3 + [False]*2, dtype=torch.bool), torch.tensor([True]*4 + [False]*1, dtype=torch.bool)])
    batched_f = torch.vmap(f, in_dims=(0,0,None))     
    batched_f(xs, masks, 1)

In [33]:
def __top_k_accuracy(
    alignement_similarity: torch.FloatTensor, mask: torch.BoolTensor, top_n:int
) -> torch.FloatTensor:
    _, indices = torch.sort(
        torch.masked_fill(
            alignement_similarity, torch.logical_not(mask), -float("inf")
        ),
        descending=True,
    )
    mask = mask.float()
    m = torch.isin(torch.arange(len(alignement_similarity)), indices[:,:top_n]).float().squeeze()
    acc = (m*mask).sum()/(mask.sum())
    return acc

In [127]:
batched_f = torch.vmap(__top_k_accuracy, in_dims=(0,0,None))  
for _ in range(1):
    xs =torch.randn((2,5,5))
    masks = torch.stack([torch.tensor([True]*3 + [False]*2, dtype=torch.bool), torch.tensor([True]*4 + [False]*1, dtype=torch.bool)])
    print(masks.sum(dim=1))
    print(batched_f(xs, masks, 3))

tensor([3, 4])
tensor([1., 1.])


/home/jlagesse/.tmp/ipykernel_277571/3988777898.py:11: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::isin.Tensor_Tensor. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at ../aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  m = torch.isin(torch.arange(len(alignement_similarity)), indices[:,:top_n]).float().squeeze()
